# CKKSEncoder (For CKKS scheme only)

In [1]:
import tenseal.sealapi as seal
import numpy as np
import util


In this example we demonstrate the Cheon-Kim-Kim-Song (CKKS) scheme for computing on encrypted real or complex numbers. We start by creating encryption parameters for the CKKS scheme. There are two important differences compared to the BFV scheme:

1. CKKS does not use the `plain_modulus` encryption parameter;
2. Selecting the `coeff_modulus` in a specific way can be very important when using the CKKS scheme. We will explain this further in the file `ckks_basics.cpp`. In this example we use `CoeffModulus::Create` to generate 5 40-bit prime numbers.

In [4]:
parms = seal.EncryptionParameters(seal.SCHEME_TYPE.CKKS)
poly_modulus_degree = 8192
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(seal.CoeffModulus.Create(poly_modulus_degree, [40, 40, 40, 40, 40 ]))

We create the SEALContext as usual and print the parameters.

In [5]:
context = seal.SEALContext(parms, True, seal.SEC_LEVEL_TYPE.TC128)
util.print_parameters(context)

/
|Encryption parameters: 
|	scheme: CKKS
|	poly_modulus_degree: 8192
|	coeff_modulus_size: 200 (40 40 40 40 40) bits


Keys are created the same way as for the BFV scheme.

In [6]:
keygen = seal.KeyGenerator(context)
secret_key = keygen.secret_key()
public_key = seal.PublicKey()
keygen.create_public_key(public_key)
relin_keys = seal.RelinKeys()
keygen.create_relin_keys(relin_keys)

We also set up an Encryptor, Evaluator, and Decryptor as usual.

In [8]:
encryptor = seal.Encryptor(context, public_key)
evaluator = seal.Evaluator(context)
decryptor = seal.Decryptor(context, secret_key)

To create CKKS plaintexts we need a special encoder: there is no other way to create them. The BatchEncoder cannot be used with the CKKS scheme. The `CKKSEncoder` encodes vectors of real or complex numbers into `Plaintext` objects, which can subsequently be encrypted. At a high level this looks a lot like what `BatchEncoder` does for the BFV scheme, but the theory behind it is completely different.

In [10]:
encoder = seal.CKKSEncoder(context)

In CKKS the number of slots is `poly_modulus_degree` / 2 and each slot encodes one real or complex number. This should be contrasted with `BatchEncoder` in the BFV scheme, where the number of slots is equal to `poly_modulus_degree` and they are arranged into a matrix with two rows.

In [11]:
slot_count = encoder.slot_count()
print('Number of slots: ' + str(slot_count))

Number of slots: 4096


We create a small vector to encode; the `CKKSEncoder` will implicitly pad it with zeros to full size (`poly_modulus_degree` / 2) when encoding.

In [12]:
input = [0.0, 1.1, 2.2, 3.3 ]
print('Input vector:')
print(input)

Input vector:
[0.0, 1.1, 2.2, 3.3]


Now we encode it with `CKKSEncoder`. The floating-point coefficients of `input` will be scaled up by the parameter `scale`. This is necessary since even in the CKKS scheme the plaintext elements are fundamentally polynomials with integer coefficients. It is instructive to think of the scale as determining the bit-precision of the encoding; naturally it will affect the precision of the result.

In CKKS the message is stored modulo `coeff_modulus` (in BFV it is stored modulo `plain_modulus`), so the scaled message must not get too close to the total size of `coeff_modulus`. In this case our `coeff_modulus` is quite large (200 bits) so we have little to worry about in this regard. For this simple example a 30-bit scale is more than enough.

In [14]:
plain = seal.Plaintext()
scale = pow(2.0, 30)
encoder.encode(input, scale, plain)

We can instantly decode to check the correctness of encoding.

In [16]:
output = encoder.decode_double(plain)
print('Decoded input vector: ')
print(output[0:8])

Decoded input vector: 
[-4.48651637212957e-08, 1.0999999741646955, 2.1999999834042967, 3.299999987597782, -2.3791869634729804e-08, -7.380506703368948e-09, 5.616646430304957e-08, -3.0054153105748065e-08]


The vector is encrypted the same way as in BFV.

In [17]:

encrypted = seal.Ciphertext()
encryptor.encrypt(plain, encrypted)

Basic operations on the ciphertexts are still easy to do. Here we square the
ciphertext, decrypt, decode, and print the result. We note also that decoding
returns a vector of full size (`poly_modulus_degree` / 2); this is because of
the implicit zero-padding mentioned above.

In [18]:
evaluator.square_inplace(encrypted)
evaluator.relinearize_inplace(encrypted, relin_keys)

We notice that the scale in the result has increased. In fact, it is now the square of the original scale: 2^60.

In [21]:
print('Scale in squared input: ' + str(encrypted.scale) + ' (' + str(np.log2(encrypted.scale)) + ' bits)')
decryptor.decrypt(encrypted, plain)
output = encoder.decode_double(plain)
print(output[0:8])

Scale in squared input: 1.152921504606847e+18 (60.0 bits)
[-5.068293803494718e-13, 1.210001494082472, 4.840001080295021, 10.890001288345392, 1.781565968336205e-11, 5.281918976196055e-13, 7.045305228866165e-13, 4.842436535973635e-14]


The CKKS scheme allows the scale to be reduced between encrypted computations. This is a fundamental and critical feature that makes CKKS very powerful and flexible. We will discuss it in great detail in `3_levels.cpp` and later in `4_ckks_basics.cpp`.